it's a small script
i fetch promtps from ed and test them here

look


In [1]:
from jinja2 import Template
from datetime import datetime, timezone
from pathlib import Path

PROMPTS_DIR = Path(".")

some random variables


In [2]:
TEMPLATE_VARS = {
    "date_and_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %A %H:%M:%S"),
    "user_conversation_language": "en",
    "relationships_disabled": False,
    "clarify_unknown_faces": True,
    "picture_present": True,
    "image_generation_available": True,
}


# code bs
def test_template(template_path: Path, batch_text: str, vars: dict) -> tuple[bool, str]:
    """Test a template and return (success, message)."""
    try:
        with open(template_path) as f:
            template_content = f.read()

        t = Template(template_content)
        result = t.render(batch_text=batch_text, **vars)
        return True, f"OK ({len(result):,} chars rendered)"
    except Exception as e:
        return False, str(e)


## Test All Templates


In [12]:
# What i test
templates_to_test = [
    ("user_bio_email_summary_prompt.jinja2", "EMAIL_BATCH_TEXT"),
    ("user_bio_email_summary_prompt_fallback.jinja2", "EMAIL_BATCH_TEXT"),
    ("user_bio_calendar_summary_prompt.jinja2", "CALENDAR_BATCH_TEXT"),
    ("user_bio_conversation_summary_prompt.jinja2", "User: Hi!\nAssistant: Hello!"),
    ("user_bio_web_crawl_summary_prompt.jinja2", "LinkedIn profile data..."),
]

print("Testing with relationships_disabled =", TEMPLATE_VARS["relationships_disabled"])
print("=" * 60)

all_passed = True
for template_name, batch_text in templates_to_test:
    template_path = PROMPTS_DIR / template_name
    if not template_path.exists():
        print(f"⚠️  {template_name}: FILE NOT FOUND")
        continue

    success, message = test_template(template_path, batch_text, TEMPLATE_VARS)
    status = "✅" if success else "❌"
    print(f"{status} {template_name}: {message}")
    if not success:
        all_passed = False

print("=" * 60)
if all_passed:
    print("All templates passed!")
else:
    print("Some templates failed - check errors above")

Testing with relationships_disabled = False
✅ user_bio_email_summary_prompt.jinja2: OK (33,008 chars rendered)
✅ user_bio_email_summary_prompt_fallback.jinja2: OK (33,011 chars rendered)
✅ user_bio_calendar_summary_prompt.jinja2: OK (22,579 chars rendered)
✅ user_bio_conversation_summary_prompt.jinja2: OK (27,752 chars rendered)
✅ user_bio_web_crawl_summary_prompt.jinja2: OK (28,748 chars rendered)
All templates passed!


## Preview Rendered Output (Email Template)


jinja errors


In [7]:
# Render and show a portion of the email template output
template_path = PROMPTS_DIR / "fte_intent_planner.jinja2"

with open(template_path) as f:
    template_content = f.read()

planner_vars = {
    # Face clarification (THE KEY ONES)
    "clarify_unknown_faces": True,
    "clarify_unknown_faces_msg": 'Could you please tell me who each person in the photo is? For example, "The person on the left is X, the person in the middle is Y, and the person on the right is Z."',
    # Basic user/bot info
    "first_name": "Test",
    "user_name": "Test",
    "bot_name": "Replika",
    "relationship_status": "friend",
    "user_pronoun": "he",
    "replika_gender": "female",
    # Context
    "turn_number": 5,
    "date": "2026-02-02 Sunday",
    "date_and_time": "2026-02-02 Sunday 12:00:00",
    "conversation_language": "en",
    # Face-related
    "unknown_faces_present": True,
    "people_detected": True,
    "picture_present": True,
    # Safety
    "is_safe": True,
    "safety_verdict": "safe",
    # Other required fields (can be empty/None)
    "user_bio": "",
    "summary": "",
    "user_message": "here's my family",
    "tools_section": "<tools>...</tools>",
    "goals": [],
    "user_interests": [],
    "replika_interests": [],
    "previous_summaries": [],
    "onboarding_survey": {},
    "onboarding_survey_string": "",
    "replika_diaries": [],
    "statements_about_user": [],
    "statements_about_replika": [],
    "image_generation_available": False,
    "has_topic_pivot": False,
    "client_has_video_fte_ready": False,
}


t = Template(template_content)
rendered = t.render(
    batch_text="EMAIL_BATCH_TEXT",
    # date_and_time=datetime.now(timezone.utc).strftime("%Y-%m-%d %A %H:%M:%S"),
    user_conversation_language="en",
    # relationships_disabled=True,
    # picture_present=True,
    # image_generation_available=True,
    **planner_vars,
)

print(rendered)

max_thinking_tokens: 1000

# Goal

**Context:**

- You are a planning agent that outputs JSON execution plans to help Replika (the conversational AI, also called Assistant) use available tools during their First Session with a new user.
- Replika builds genuine connection through intellectual compatibility, shared humor, aesthetic appreciation, and emotional resonance.
- Enhance conversations when helpful—never interrupt natural flow.














**Your Functions:**

- **FUNCTION 2: WEB SEARCH** - Decide when to search for current information

- **FUNCTION 4: TASTE INTEGRATION** - Add Replika's personality from <your_taste> when relevant
- **FUNCTION 6: PICTURE REQUESTS** - Ask for photos when users mention people/pets/activities




# General JSON plan structure and Available Tools

## JSON plan structure

Each step is a JSON object with four keys:

- `identity` (string): Unique step ID (e.g., "search_relevant_info"). Used for dependencies.
- `tool_name` (string): Tool to execute (f

## Find Jinja Syntax Issues


In [ ]:
import re


def find_jinja_issues(template_path: Path) -> list[tuple[int, str, str]]:
    """Find potential Jinja syntax issues in a template."""
    issues = []

    with open(template_path) as f:
        lines = f.readlines()

    # Track if/endif balance
    if_stack = []

    for i, line in enumerate(lines, 1):
        # Check for common typos
        if "{% if endif %}" in line:
            issues.append((i, line.strip(), "Typo: 'if endif' should be just 'endif'"))

        if "{% endif if %}" in line:
            issues.append((i, line.strip(), "Typo: 'endif if' is invalid"))

        # Track if/endif
        if re.search(r"\{%\s*if\s+", line):
            if_stack.append(i)
        if re.search(r"\{%\s*endif\s*%\}", line):
            if if_stack:
                if_stack.pop()
            else:
                issues.append((i, line.strip(), "Unmatched endif (no opening if)"))

    # Check for unclosed ifs
    for if_line in if_stack:
        issues.append((if_line, lines[if_line - 1].strip(), "Unclosed if block"))

    return issues


# Scan all templates
print("Scanning for Jinja syntax issues...\n")

for template_file in PROMPTS_DIR.glob("*.jinja2"):
    issues = find_jinja_issues(template_file)
    if issues:
        print(f"❌ {template_file.name}:")
        for line_num, line, issue in issues:
            print(f"   Line {line_num}: {issue}")
            print(
                f"   Content: {line[:80]}..."
                if len(line) > 80
                else f"   Content: {line}"
            )
    else:
        print(f"✅ {template_file.name}: No issues found")